In [40]:
# ============================================
# YouTube Emotion Scraper (WORKING IN 2025)
# Tanpa API Key • Tanpa yt-dlp Search
# ============================================

import requests
import re
import csv
import itertools
from bs4 import BeautifulSoup
import yt_dlp

In [41]:
# ------------------------------
# 1. HTML SEARCH (no API key)
# ------------------------------

def search_youtube(keyword, max_results=40):
    query = keyword.replace(" ", "+")
    url = f"https://www.youtube.com/results?search_query={query}"

    print(f"Searching YouTube: {keyword}")

    try:
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        html = r.text

        # Extract video IDs from HTML
        video_ids = re.findall(r"watch\?v=([a-zA-Z0-9_-]{11})", html)

        # Remove duplicates
        unique_ids = []
        seen = set()

        for vid in video_ids:
            if vid not in seen:
                seen.add(vid)
                unique_ids.append(vid)

        print(f" -> Found {len(unique_ids)} videos")

        return unique_ids[:max_results]

    except Exception as e:
        print("Search error:", e)
        return []


In [42]:
# ------------------------------
# 2. SCRAPE COMMENTS
# ------------------------------

def scrape_comments(video_id, max_comments=700):
    url = f"https://www.youtube.com/watch?v={video_id}"

    ydl_opts = {
        "quiet": True,
        "skip_download": True,
        "extract_flat": True,
        "getcomments": True,
    }

    print(f"   Scraping comments from video {video_id} ...")

    comments = []

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)

        if 'comments' in info:
            for c in info['comments']:
                if "text" in c:
                    comments.append(c["text"])

                if len(comments) >= max_comments:
                    break

    except Exception as e:
        print(f"   Error scraping comments for {video_id}: {e}")

    print(f"   -> Collected {len(comments)} comments")
    return comments

In [43]:
# ------------------------------
# 3. SAVE CSV
# ------------------------------

def save_csv(filename, rows):
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["comment"])
        writer.writerows([[r] for r in rows])
    print(f"Saved {len(rows)} comments to {filename}")

In [44]:
# ------------------------------
# 4. EMOTION KEYWORDS
# ------------------------------

emotion_keywords = {
    "sadness": ["sad song", "broken heart", "sad story", "emotional music"],
    "anger": ["angry rant", "rage compilation", "angry moments"],
    "fear": ["scary story", "horror story", "creepy videos"],
    "joy": ["happy moments", "funny video", "joyful moments"],
    "love": ["love song", "romantic story", "relationship advice"]
}

In [45]:
# ------------------------------
# 5. MASTER FUNCTION
# ------------------------------

def collect_emotion_comments(emotion, keywords, target_count=5000):
    print(f"\n============================")
    print(f" Collecting EMOTION: {emotion}")
    print(f"============================")

    # 1. Search videos
    video_ids = list(
        itertools.chain.from_iterable(
            search_youtube(f"{kw} english comments") for kw in keywords
        )
    )

    print(f"Total videos found: {len(video_ids)}")

    if not video_ids:
        print("No videos found. Skipped.")
        return

    all_comments = []

    # 2. Scrape comments from each video
    for vid in video_ids:
        comments = scrape_comments(vid, max_comments=700)
        all_comments.extend(comments)

        if len(all_comments) >= target_count:
            break

    # 3. Save results
    save_csv(f"{emotion}.csv", all_comments[:target_count])

In [46]:
# ============================================
# RUN ALL EMOTIONS
# ============================================

for emotion, keywords in emotion_keywords.items():
    collect_emotion_comments(emotion, keywords, target_count=5000)

print("\n=== ALL DONE! ===")


Searching YouTube: sad song english comments
 -> Found 41 videos
Searching YouTube: broken heart english comments
 -> Found 27 videos
Searching YouTube: sad story english comments
 -> Found 38 videos
Searching YouTube: emotional music english comments
 -> Found 34 videos
Total videos found: 139
   Scraping comments from video pV8vj17LTlE ...


   -> Collected 700 comments
   Scraping comments from video 856iH4njBfg ...


   -> Collected 700 comments
   Scraping comments from video TMhWQeka5Lw ...


   -> Collected 700 comments
   Scraping comments from video eCKE87mbC_8 ...


   -> Collected 700 comments
   Scraping comments from video dPPoh33xd_I ...


...

   -> Collected 270 comments
   Scraping comments from video PibeAW9lNvA ...


   -> Collected 43 comments
   Scraping comments from video 48c4zHrsVQ8 ...


   -> Collected 2 comments
   Scraping comments from video 5gljAOM5zsQ ...


   -> Collected 171 comments
   Scraping comments from video KMNyDlpX5Zg ...


   -> Collected 700 comments
Saved 5000 comments to love.csv

=== ALL DONE! ===


In [ ]:
# Ini keyword ee

# Uncomment kalo mau jalanin scrap
# emotion = "sadness"
# emotion = "anger"
# emotion = "fear"
# emotion = "joy"
# emotion = "love"

# queries = {
#     "sadness":  '"I feel sad" OR "I am sad" OR "so sad" OR "heartbroken" OR "feeling down" OR "I feel depressed"',
#     "anger":    '"I am angry" OR "so mad" OR "pissed off" OR "furious" OR "so angry" OR "I hate this"',
#     "fear":     '"I am scared" OR "I feel anxious" OR "terrified" OR "I am afraid" OR "panic attack"',
#     "joy":      '"I feel happy" OR "so happy" OR "excited" OR "grateful" OR "I am joyful" OR "good news"',
#     "love":     '"I love you" OR "I love this" OR "so in love" OR "I adore" OR "I really love"'
# }
# keyword = queries[emotion]